In [1]:
import os
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import timm
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
batch_size = 50  # Reduced batch size
epochs = 5
learning_rate = 1e-4
num_classes = 101

# Data transforms for Food101
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load the already downloaded dataset (no download=True)
train_dataset = datasets.Food101(root='data', split='train', transform=transform, download=False)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Model, loss, optimizer
model = timm.create_model('efficientnet_b0', pretrained=False, num_classes=num_classes)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scaler = GradScaler()

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for images, labels in tqdm(train_loader, desc=f"Epoch [{epoch+1}/{epochs}]"):
        images, labels = images.to(device), labels.to(device)

        # Mixed precision context
        with autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Save model
torch.save(model.state_dict(), 'efficientnet_b0_no_kd_food101.pth')

/root/miniconda3/envs/dl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_156228/3612883619.py:37: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch [1/5]:   0%|          | 0/1515 [00:00<?, ?it/s]/tmp/ipykernel_156228/3612883619.py:49: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch [1/5]: 100%|██████████| 1515/1515 [39:03<00:00,  1.55s/it]


Epoch [1/5], Loss: 4.4908, Accuracy: 3.07%


Epoch [2/5]: 100%|██████████| 1515/1515 [32:05<00:00,  1.27s/it]


Epoch [2/5], Loss: 4.0388, Accuracy: 9.04%


Epoch [3/5]: 100%|██████████| 1515/1515 [33:03<00:00,  1.31s/it]


Epoch [3/5], Loss: 3.7345, Accuracy: 14.00%


Epoch [4/5]: 100%|██████████| 1515/1515 [32:55<00:00,  1.30s/it]


Epoch [4/5], Loss: 3.4614, Accuracy: 18.74%


Epoch [5/5]: 100%|██████████| 1515/1515 [32:58<00:00,  1.31s/it]


Epoch [5/5], Loss: 3.2218, Accuracy: 23.07%
